In [11]:
import pandas as pd
import numpy as np
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, coo_matrix
from sklearn.cluster import MiniBatchKMeans

In [12]:
data = []

with open('cosmo_content_storage_final_cut.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))

In [32]:
df = pd.DataFrame.from_records(data)
df

,url,doc_id,description,ts,title
0,https://vz.ru/news/2019/12/6/1012187.html,1000029981939875422,У России есть необходимые конкурентные преимущ...,1575625264,Новак заявил о способности России стать лидеро...
1,https://meduza.io/news/2019/12/06/sk-otchitals...,1000103932845397396,По делу об убийстве главы ингушского центра «Э...,1575634419,"СК отчитался о поимке банды, причастной к убий..."
2,http://www.vestikavkaza.ru/material/283355,1000115462666782749,Президент Азербайджана Ильхам Алиев и первый в...,1575447101,Ильхам Алиев и Мехрибан Алиева посетили выстав...
3,http://krizis-kopilka.ru/archives/70325,1000129039443894284,Проблемы и вызовы языковой политики анализируе...,1575767444,Язык и идеология
4,https://radiosputnik.ria.ru/20191203/156189583...,1000131983536261699,Отношения России и Турции не являются альтерна...,1575359865,Эрдоган: отношения Турции с РФ не альтернативн...
...,...,...,...,...,...
88363,https://lenta.ru/news/2019/12/05/money/,999910919457925598,Производитель электроники Xiaomi запустил прил...,1575559155,Xiaomi начала выдавать микрокредиты
88364,https://megaobzor.com/Xbox-One-otlichno-prodal...,999919369677233771,Многие пользователи специально ждут «Черную пя...,1575391338,Xbox One отлично продалась с Черную пятницу
88365,https://car.ru/news/automobili/59574-kseniyu-b...,999942425463634000,Ксению Бородину раздражают хамы на дорогах,1575543490,Ксению Бородину раздражают хамы на дорогах
88366,https://www.livesport.ru/football/2019/12/05/m...,99994363695107503,Главный тренер сборной Бельгии поделился мнени...,1575636048,Наставник сборной Бельгии считает Италию фавор...


In [14]:
tfidf = TfidfVectorizer()
description = tfidf.fit_transform(df["description"].fillna(""))
title = tfidf.fit_transform(df["title"].fillna(""))
url = tfidf.fit_transform(df["url"].fillna(""))

In [15]:
train = hstack([description, title, url, coo_matrix(df['ts'].values).T])

In [18]:
km = MiniBatchKMeans(n_clusters=300, init='k-means++', n_init=1,
                         init_size=1000, batch_size=1000)

In [19]:
km.fit(train)

MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
                init_size=1000, max_iter=100, max_no_improvement=10,
                n_clusters=300, n_init=1, random_state=None,
                reassignment_ratio=0.01, tol=0.0, verbose=0)

In [28]:
df.shape

(88368, 5)

In [29]:
km.predict(train).shape

(88368,)

In [33]:
answer = pd.DataFrame(columns=['doc_id', 'cat'])
answer['doc_id'] = df['doc_id']
answer['cat'] = km.predict(train)

In [34]:
answer.to_csv('sub.txt', header=True, index=False)